<a href="https://colab.research.google.com/github/dgmyaing/DS2002/blob/main/DS2002API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install yahooquery
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
from yahooquery import Ticker
#don't think i needed to install yahooquery but just in case I did


In [10]:
#get stock from the user
#It was asking me to put credit card info in...
stock=input()


apikey="ihBCrKGRAG2OY2LkqzuP88zBQjH8x4IQ1XCW2FYQ"

url = "https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols=AAPL%2CBTC-USD%2CEURUSD%3DX"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)


#that's pretty ugly. we can do this
print(json.dumps(response.json(), indent=4))

MSFT
{"message":"Forbidden","hint":"Sign up for API key https://financeapi.net/tutorial"}
{
    "message": "Forbidden",
    "hint": "Sign up for API key https://financeapi.net/tutorial"
}


In [ ]:
#Having trouble getting output from API still (asking for my credit card info to make an account).
#That being said, this code should still work for other parameters.
def fetch_basic_stock_data(symbols):
    stocks = Ticker(symbols)
    summary = stocks.summary_detail
    company_names = stocks.quote_type

    for symbol in symbols.split():
        try:
            ticker = symbol.upper()
            company = company_names[ticker]['longName']
            price = summary[ticker]['regularMarketPrice']
            print(f"Stock Ticker: {ticker}, Company: {company}, Current Market Price: ${price}")
        except KeyError:
            print(f"Error fetching data for {symbol}")

In [ ]:
def fetch_additional_stock_data(symbols, module):
    stocks = Ticker(symbols)
    data = stocks.asset_profile if module == 'ROA' else stocks.summary_detail

    stock_info = []
    for symbol in symbols.split():
        try:
            ticker = symbol.upper()
            if module == '52 Week High':
                value = data[ticker]['fiftyTwoWeekHigh']
            elif module == '52 Week Low':
                value = data[ticker]['fiftyTwoWeekLow']
            elif module == 'ROA':
                value = data[ticker]['returnOnAssets']
            else:
                value = 'Invalid Module'
            stock_info.append([ticker, module, value])
        except KeyError:
            stock_info.append([symbol, module, 'N/A'])

    df = pd.DataFrame(stock_info, columns=['Stock Ticker', 'Metric', 'Value'])
    print(df)

In [ ]:
def fetch_trending_stocks():
    trending = Ticker('SPY').trending_tickers['quotes']
    trending_data = []
    for stock in trending[:5]:  # Limiting to 5 trending stocks for ease
        try:
            ticker = stock['symbol']
            name = stock['shortName']
            price = stock['regularMarketPrice']
            high_52 = stock['fiftyTwoWeekHigh']
            low_52 = stock['fiftyTwoWeekLow']
            trending_data.append([ticker, name, price, high_52, low_52])
        except KeyError:
            continue

    df = pd.DataFrame(trending_data, columns=['Ticker', 'Company', 'Price', '52W High', '52W Low'])
    print(df)

In [ ]:
def fetch_elon_plane():
    url = "https://opensky-network.org/api/states/all"
    response = requests.get(url).json()
    for plane in response['states']:
        if plane[1] and "ElonJet" in plane[1]:  # Assuming 'ElonJet'is used, kinda confused on where name would be found
            print(f"Elon's Plane is at Latitude: {plane[6]}, Longitude: {plane[5]}")
            return
    print("Elon's plane not found")

if __name__ == "__main__":
    symbols = input("Enter stock symbols separated by space: ")
    fetch_basic_stock_data(symbols)

    module = input("Choose a module (52 Week High, 52 Week Low, ROA): ")
    fetch_additional_stock_data(symbols, module)

    fetch_trending_stocks()
    fetch_elon_plane()